#This is an email reader created by Ryan Smith

##This will allow for constant monitoring of the emails without manual intervention

V1 release notes

contains the following changes and updates
- uses psutil to verify outlook instance is running (new)
- email address exact match (ghost emails - prod/QA issue)(new)
- uses the properties from v0.4
-- connects to outlook
-- verifies the inbox
-- validates the unreads
-- checks for matching instances not exact matches
-- connects and reads the data (test data emails)
-- places each file email in a seperate csv
-- downloads attachment to specified file

In [12]:
import win32com.client
import os
import pandas as pd
import csv
from datetime import datetime
import sched, time
import psutil
import zipfile
import re

In [2]:
def is_outlook_running():
    """This verifies that outlook is actually running and creates a new instance if it isnt"""
    for p in psutil.process_iter(attrs=['pid', 'name']):
        if "OUTLOOK.EXE" in p.info['name']:
            print("Yes", p.info['name'], "is running")
            break
    else:
        print("No, Outlook is not running")
        os.startfile("outlook")
        print("Outlook is starting now...")

In [3]:
def body_csv():
    """Decorator coming in future release this scans the body of the email to create a csv file containing the correct 
    details it verifies againt the email address and the subject line"""
    #connecting to email program
    M = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
    
    #blank list to append to
    headings = []
    i = []
    len_i = []
    #access inbox
    inbox_a = M.Folders('ryansmith120@hotmail.com').Folders('Inbox')
    messages = inbox_a.Items

    #create for loop to search for unread messages
    for message in messages:
        if "This is a csv test".lower() in message.subject.lower() and message.Unread == True:
            details = re.split(':| |\r|\n',(message.body))
            i.append([details[7],details[10],details[13]])
            headings.append([details[6],details[9],details[12]])
            len_i = [x for x in range(len(i))]
            message.Unread = False
        
        elif message.Class == 43:
            if message.SenderEmailAddress == 'ryansmith120@hotmail.com' and message.Unread == True:
                i.append((message.body).split()[4])
                len_i = [x for x in range(len(i))]
                message.Unread = False
        
        #create an exclusion for QA environment
    
    def new_csv(i,len_i):
        while len(i) > 0:
            a = datetime.today()
            date = int(a.strftime('%Y%m%d%H%M%S'))
            file_name ="csv test{} {}.csv".format(len_i[0],date)
            
            #create the file for each email
            file_out = open(os.getcwd() + '\\' + file_name, 'w',newline = '')
            #add headers
            with open(file_name, 'a') as f:
                info = csv.writer(f)
                info.writerow([headings[0]])
                info.writerow([i[0]])
                f.close()
            #push to correct file path
            headings.pop(0)
            i.pop(0)
            len_i.pop(0)
    #run the writer for a unique file for each item
    new_csv(i,len_i)

In [4]:
def attach_download():
    """This verfies an attachment file and downloads it to a predetermined folder
    it verifies email address as well as subject"""
    M = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")

    inbox_a = M.Folders('ryansmith120@hotmail.com').Folders('Inbox')
    messages = inbox_a.Items
    
    for message in messages:
        
        if "This is an attachment test".lower() in message.subject.lower() and message.Unread == True:
            attachment = message.Attachments
            for attachment in message.Attachments:
                attachment_name = str(attachment).lower()
                attachment.SaveAsFile(os.getcwd() + '\\' + attachment_name)
                message.Unread = False
        
        #unzip file with password
#         with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
#             zip_ref.extractall(pwd = bytes('xxxxx','utf-8'), directory_to_extract_to)
        
        elif message.Class == 43:
            if message.SenderEmailAddress == 'ryan.james.smith611@gmail.com' and message.Unread == True:
                attachment = message.Attachments
                for attachment in message.Attachments:
                    attachment_name = str(attachment).lower()
                    attachment.SaveAsFile(os.getcwd() + '\\' + attachment_name)
                    message.Unread = False

In [14]:
def execute():
    """this executes the reader"""
    while True:
        is_outlook_running()
        body_csv()
        attach_download()
        time.sleep(600)

In [ ]:
execute()

Yes OUTLOOK.EXE is running
['Bla', 'bla', 'bla', '', '', '', 'Name', 'Ryan', '', 'Surname', 'Smith', '', 'ID', '123456', '', '', '', '', '', '', '', '', '', 'Sent', 'from', 'my', 'Galaxy', '', '']
[['Name', 'Surname', 'ID']]
